<a href="https://colab.research.google.com/github/jmcaussade/Homologacion-Car-Makers/blob/main/Estandarizacion_marcas_modelos_probabilidades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.4 MB/s eta 0:00:00


In [6]:
import pandas as pd
import re
import os
from rapidfuzz import process, fuzz

In [7]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
from google.colab import files

In [56]:
ruta_data_set = '/content/drive/MyDrive/Xinergy/Proyecto_homologacion_autos/database.xlsx'
df = pd.read_excel(ruta_data_set, sheet_name='Anexo 6')
df.head()
df

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg
0,NaN,355845,Auto,2023-09-01 00:01:13,SZD861A,3HGCM66563G001615,2003.0,estandar,Honda,ACCORD,Auto,NaN
1,NaN,355847,Camion,2023-09-01 00:06:09,TP9498G,3N6DD25T2EK005444,2014.0,estandar,NISSAN,ESTACAS,camion,NaN
2,NaN,355849,Auto,2023-09-01 00:08:38,NCZ1892,MEX5G2603JT120368,2018.0,estandar,Volkswagen,VENTO,Auto,NaN
3,NaN,355857,Auto,2023-09-01 00:24:34,FCJ597B,1C4BJWEG6FL629655,2015.0,estandar,JEEP,J WRANGLER,Auto,NaN
4,NaN,355864,Auto,2023-09-01 00:46:09,NJA2747,MA6CB6CD6KT029772,2019.0,estandar,CHEVROLET,BEAT,Auto,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
279424,NaN,745808,Equipo Ligero,2024-08-31 00:00:00,TDY040B,3N1CN7AD7KK399688,2019.0,NaN,NISSAN,VERSA,Equipo Ligero,NaN
279425,NaN,745810,Equipo Ligero,2024-08-31 00:00:00,JUB4510,LSJA24U99RN010717,2024.0,NaN,MG,HS,Equipo Ligero,NaN
279426,NaN,745811,Equipo Ligero,2024-08-31 00:00:00,RTR920A,3G1SF21X97S112886,2007.0,NaN,CHEVROLET,CHEVY,Equipo Ligero,NaN
279427,NaN,745814,Equipo Ligero,2024-08-31 00:00:00,PWEE63E,LJD5AA194R0124080,2024.0,NaN,KIA,SELTOS,Equipo Ligero,NaN


In [57]:
car_makers = set([
    "acura", "alfa romeo", "audi", "baic", "bajaj", "bentley", "bmw", "buick",
    "byd", "cadillac", "chevrolet", "changan", "chery", "chrysler", "cupra", "daewoo",
    "dodge", "ducati", "fiat", "ford", "foton", "freightliner", "gmc", "great wall",
    "haval", "harley", "hino", "honda", "hummer", "hyundai", "infiniti", "isuzu",
    "italika", "jac", "jaguar", "jeep", "jmc", "kenworth", "kia", "kurazai",
    "land rover", "lexus", "lincoln", "mack", "masserati", "mazda", "mercedes-benz", "mercedes benz",
    "mg", "mini", "mitsubishi", "nissan", "peugeot", "peterbilt", "piaggio", "pontiac",
    "porsche", "ram", "renault", "saab", "scania", "seat", "sinotruk", "subaru",
    "suzuki", "tesla", "tiggo", "toyota", "volkswagen", "volvo", "yamaha", "vento",
    "kawazaki", "ktm", "aprilia", "bimota", "mv agusta", "royal enfield",
    "harley-davidson", "hero", "husqvarna", "indian", "vespa", "zanella", "benelli",
    "cf moto", "kymco", "norton", "sym", "lotus", "citroen", "dacia", "opel", "skoda",
    "smart", "tata", "mahindra", "geely", "proton", "lada", "ssangyong", "maruti",
    "chrysler", "saturn", "daewoo", "saic", "zhongxing", "haima", "reno", "vw", "ram"
])


In [58]:
df["clean_car_brand"] = (
    df["car_brand"]
    .fillna("")  # Replace NaN with an empty string
    .astype(str)  # Ensure all values are strings
    .str.lower()  # Convert all strings to lowercase
    .str.replace(r'[^a-z0-9 ]', '', regex=True)  # Remove only special characters
)

In [59]:
# Function to preprocess the description and match car makers
def preprocess_description(description, car_makers):
    tokens = re.findall(r'\w+', description.lower())  # Tokenize into words
    if len(tokens) == 1:  # If it's a single word, proceed with direct matching
        if tokens[0] in car_makers:
            return tokens[0]
    # If multiple words, filter matching words from the tokens
    potential_matches = [token for token in tokens if token in car_makers]
    return potential_matches[0] if potential_matches else description  # Return first match or full string


# Function to get best match using fuzzy matching
def get_best_match(brand, car_makers):
    cleaned_brand = preprocess_description(brand, car_makers)  # Get cleaned brand
    if cleaned_brand in car_makers:  # Direct match for single word
        return cleaned_brand, 100  # Exact match
    # If no exact match, perform fuzzy matching for single or multi-word brands
    match = process.extractOne(cleaned_brand, car_makers, scorer=fuzz.token_set_ratio)
    return match[0], match[1]

In [60]:
df["best_match"], df["similarity"] = zip(*df["clean_car_brand"].apply(lambda x: get_best_match(x, car_makers)))

In [62]:
df

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg,clean_car_brand,best_match,similarity
0,NaN,355845,Auto,2023-09-01 00:01:13,SZD861A,3HGCM66563G001615,2003.0,estandar,Honda,ACCORD,Auto,NaN,honda,honda,100.0
1,NaN,355847,Camion,2023-09-01 00:06:09,TP9498G,3N6DD25T2EK005444,2014.0,estandar,NISSAN,ESTACAS,camion,NaN,nissan,nissan,100.0
2,NaN,355849,Auto,2023-09-01 00:08:38,NCZ1892,MEX5G2603JT120368,2018.0,estandar,Volkswagen,VENTO,Auto,NaN,volkswagen,volkswagen,100.0
3,NaN,355857,Auto,2023-09-01 00:24:34,FCJ597B,1C4BJWEG6FL629655,2015.0,estandar,JEEP,J WRANGLER,Auto,NaN,jeep,jeep,100.0
4,NaN,355864,Auto,2023-09-01 00:46:09,NJA2747,MA6CB6CD6KT029772,2019.0,estandar,CHEVROLET,BEAT,Auto,NaN,chevrolet,chevrolet,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279424,NaN,745808,Equipo Ligero,2024-08-31 00:00:00,TDY040B,3N1CN7AD7KK399688,2019.0,NaN,NISSAN,VERSA,Equipo Ligero,NaN,nissan,nissan,100.0
279425,NaN,745810,Equipo Ligero,2024-08-31 00:00:00,JUB4510,LSJA24U99RN010717,2024.0,NaN,MG,HS,Equipo Ligero,NaN,mg,mg,100.0
279426,NaN,745811,Equipo Ligero,2024-08-31 00:00:00,RTR920A,3G1SF21X97S112886,2007.0,NaN,CHEVROLET,CHEVY,Equipo Ligero,NaN,chevrolet,chevrolet,100.0
279427,NaN,745814,Equipo Ligero,2024-08-31 00:00:00,PWEE63E,LJD5AA194R0124080,2024.0,NaN,KIA,SELTOS,Equipo Ligero,NaN,kia,kia,100.0


In [63]:
# Count results above and below similarity threshold
count_above_70 = (df["similarity"] >= 70).sum()
count_below_70 = (df["similarity"] < 70).sum()

In [64]:
print(f"Above 70: {count_above_70}, Below 70: {count_below_70}")

Above 70: 230982, Below 70: 48447


#Descarga archivo


In [16]:
#Descarga en Colab session
file_name = 'my_dataframe.xlsx'
df.to_excel(file_name, index=False)

In [17]:
#Para descargar localmente
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>